In [73]:
import pandas as pd
import sqlparse
import re
import json 

data=pd.read_csv('jbqry100.csv',quotechar="\"")
table_info_csv=pd.read_csv('vnet_schema_information.csv')
table_info_csv['table_name_lower']=table_info_csv['table_name'].str.lower()
table_info_csv['column_name']=table_info_csv['column_name'].str.lower()
table_column = table_info_csv.groupby('table_name_lower')['column_name'].apply(list).to_dict()
table_name = {table.lower():table for table in table_info_csv['table_name'].unique()}

In [74]:
def sql_parse(sql):
    keywords=['from','into','using', 'join', 'left join',
              'right join', 'left outer join', 'right outer join',
             'inner join']
    
    parsed_sql=[('','','')]
    tables, columns, marked_phrase= [],[],[]
    
    try:
        statement = sqlparse.parse(sql)[0]
    except:
        return
    #This will parse the query into tokens
    tokens=[token for token in statement.tokens if str(token.ttype)!='Token.Text.Whitespace']
    for i,token in enumerate(tokens):
        #If current token is a keyword and the next token is a sub query (start with "(" ) then parse the next token as a SQL query
        if token.value.lower() in keywords:
            if tokens[i+1].value[0]=='(':
                sub_query=sql_parse(tokens[i+1].value[1 : tokens[i+1].value.rfind(')')])
                tables.extend(sub_query[0])
                marked_phrase.extend(sub_query[1])
            else:
                tables.append(tokens[i+1].value)
        
        #If token is after a "ON", "AND", "OR" keyword, append it to marked phrase, if it's "WHERE", parse the phrase after "WHERE" as a query
        if tokens[i-1].value.lower() in ['on','and','or']:
                marked_phrase.append(token.value)
        elif token.value.lower().split(' ')[0]=='where' :
            sub_query=sql_parse(token.value[6:])
            tables.extend(sub_query[0])
            marked_phrase.extend(sub_query[1])
      
    #Get the table name only
    tables=[table.split(' ')[0].lower() for table in tables]    
    
    #Extracted columns pair from marked phrases
    for j in marked_phrase:
        try:
            columns.extend([[column.split('.')[-1] for column in pair] for pair in re.findall('([a-zA-Z\_\.]+)\s*(?:=|<=|>=|<>|>|<)\s*([a-zA-Z\_\.]+)',j)])
        except:
            continue
            
    return tables, marked_phrase, columns
            
def relationship_extraction(sql):
#     sql=data.loc[13,'query_text']
    parsed_sql = sql_parse(sql)
    relating_column=[]
    
    #Get tables that appear in the schema only
    tables=[table for table in parsed_sql[0] if table in table_name.keys()]
    
    
    for i,(c1,c2) in enumerate(parsed_sql[2]):
        check1=None
        check2=None
        for table in tables:
            if check1==None and c1.lower() in table_column[table]:
                check1=table
            if c2.lower() in table_column[table] and check1!=table:
                check2=table
        if check1!=None and check2!=None:
            relating_column.append((c1, c2, check1+'@'+check2))
      
    #Create a relationship dictionary that contain tables key
    relationship={}
    for table in tables:
        relationship[table]={}
        for another_table in tables:
            if table!=another_table:
                relationship[table][another_table]={}
    
    # Generate the relationship value for the dictionary
    for c1,c2,table_pair in relating_column:
        relationship[table_pair.split('@')[0]][table_pair.split('@')[1]][c1]=c2
        relationship[table_pair.split('@')[1]][table_pair.split('@')[0]][c2]=c1

    #This will remove all empty relationship in the dictionary
    output={m:{k:relationship[m][k] for k in relationship[m].keys() if relationship[m][k]!={}} for m in relationship.keys() if {k:relationship[m][k] for k in relationship[m].keys() if relationship[m][k]!={}}!={}}
    return output

In [76]:
sql=data['query_text'][13]
result=relationship_extraction(sql)
print(json.dumps(result, indent=4, sort_keys=True))
# sql_parse(sql)

{
    "inventoryfactcurrent": {
        "rangederived": {
            "invc_product_ID": "rngd_product_ID",
            "invc_retail_ID": "rngd_retail_ID",
            "invc_store_ID": "rngd_store_ID"
        }
    },
    "rangederived": {
        "inventoryfactcurrent": {
            "rngd_product_ID": "invc_product_ID",
            "rngd_retail_ID": "invc_retail_ID",
            "rngd_store_ID": "invc_store_ID"
        }
    }
}


In [15]:
import pandas as pd
import xml.etree.ElementTree as ET
import re
# from merdeep import merge

df = pd.read_csv ('jbqry100.csv')
xml=df['query_plan'][50].replace('xmlns="http://schemas.microsoft.com/sqlserver/2004/07/showplan" ','')

In [16]:
def merge(a, b, path=None):
    "merges b into a"
    if path is None: path = []
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge(a[key], b[key], path + [str(key)])
            elif a[key] == b[key]:
                pass # same leaf value
            else:
                raise Exception('Conflict at %s' % '.'.join(path + [str(key)]))
        else:
            a[key] = b[key]
    return a
# from merdeep import merge 

def xml_relationship_parsing(xml):
    tree = ET.ElementTree(ET.fromstring(xml))
    root = tree.getroot()[0][0]
    df_column = pd.DataFrame(columns=["Database", "Schema", "Table", "Column"])
    empty_df = pd.DataFrame(columns=["Database", "Schema", "Table", "Column"],data=[['________']*4])
    t1=[[c1.attrib,c2.find('.//ColumnReference').attrib] for a in root.findall('.//SeekPredicates') for c1,c2 in zip(a.findall('.//RangeColumns/ColumnReference'),a.findall('.//RangeExpressions//ScalarOperator')) if c2[0].tag!='Const']
    t2=[[b.attrib for b in a.findall('.//ColumnReference')] for a in root.findall('.//ProbeResidual//Compare')+root.findall('.//Residual//Compare')]
    t=t1+t2
    relationship={}
    for column1,column2 in t:
        try:
            if column1['Schema'] =='[jbhardware]' and column2['Schema'] =='[jbhardware]':
                current_relationship={
                    column1['Table'][1:-1]:{
                        column2['Table'][1:-1]:{
                            column1['Column']:column2['Column']
                        }
                    },
                    column2['Table'][1:-1]:{
                        column1['Table'][1:-1]:{
                            column2['Column']:column1['Column']
                        }
                    }
                }
                relationship=merge(relationship, current_relationship, path=None)
    #                 relationship[column1['Table'][1:-1]][column2['Table'][1:-1]][column1['Column']]=column2['Column']
    #                 relationship[column2['Table'][1:-1]][column1['Table'][1:-1]][column2['Column']]=column1['Column']
        except:
            continue
    return relationship
#     return t

result = xml_relationship_parsing(xml)
print(json.dumps(result, indent=4, sort_keys=True))


{
    "inventoryFactCurrent": {
        "retailProduct": {
            "invc_product_ID": "rtlprd_product_ID"
        }
    },
    "pricing": {
        "product": {
            "prc_product_ID": "prd_id"
        }
    },
    "product": {
        "pricing": {
            "prd_id": "prc_product_ID"
        },
        "productFamilyGroupDetail": {
            "prd_family_ID": "pfgd_family_ID"
        },
        "retailProduct": {
            "prd_id": "rtlprd_product_ID"
        }
    },
    "productFamilyGroup": {
        "productFamilyGroupDetail": {
            "pfg_ID": "pfgd_group_ID"
        }
    },
    "productFamilyGroupDetail": {
        "product": {
            "pfgd_family_ID": "prd_family_ID"
        },
        "productFamilyGroup": {
            "pfgd_group_ID": "pfg_ID"
        }
    },
    "retailProduct": {
        "inventoryFactCurrent": {
            "rtlprd_product_ID": "invc_product_ID"
        },
        "product": {
            "rtlprd_product_ID": "prd_id"
        

In [ ]:
def xml_relationship_parsing(xml):
    tree = ET.ElementTree(ET.fromstring(xml))
    root = tree.getroot()[0][0]
    tag_list = [a for a in root.findall('.//ProbeResidual/ScalarOperator')]+[a for a in root.findall('.//Residual/ScalarOperator')]
    df_column = pd.DataFrame(columns=["db", "schema", "table", "column"])
    empty_df = pd.DataFrame(columns=["db", "schema", "table", "column"],data=[['________']*4])

    for i in range(len(tag_list)):
        phrase = tag_list[i].attrib['ScalarString'].split('=')
        column1 = [item[1:-1] for item in  phrase[0].split(' as ')[0].split('.')]
        column2 = [item[1:-1] for item in  phrase[1].split(' as ')[0].split('.')]
        df_column = pd.concat([df_column,empty_df,pd.DataFrame(columns=["db", "schema", "table", "column"], data=[column1,column2])])
    return df_column


xml_relationship_parsing(xml)

In [33]:
def xml_relationship_parsing(xml):
    tree = ET.ElementTree(ET.fromstring(xml))
    root = tree.getroot()[0][0]
    df_column = pd.DataFrame(columns=["Database", "Schema", "Table", "Column"])
    empty_df = pd.DataFrame(columns=["Database", "Schema", "Table", "Column"],data=[['________']*4])
    t1=[[c1.attrib,c2.find('.//ColumnReference').attrib] for a in root.findall('.//SeekPredicates') for c1,c2 in zip(a.findall('.//RangeColumns/ColumnReference'),a.findall('.//RangeExpressions//ScalarOperator')) if c2[0].tag!='Const']
    t2=[[b.attrib for b in a.findall('.//ColumnReference')] for a in root.findall('.//ProbeResidual//Compare')+root.findall('.//Residual//Compare')]
    t=t1+t2
    for i in range(len(t)):
    #     phrase = tag_list[i].attrib['ScalarString'].split('=')
    #     column1 = [item[1:-1] for item in  phrase[0].split(' as ')[0].split('.')]
    #     column2 = [item[1:-1] for item in  phrase[1].split(' as ')[0].split('.')]
        df_column = pd.concat([df_column,empty_df,pd.DataFrame(columns=["Database", "Schema", "Table", "Column"], data=t[i])])
    return df_column

xml_relationship_parsing(xml)

,Database,Schema,Table,Column
0,________,________,________,________
0,[inventory_new],[jbhardware],[inventoryFactCurrent],invc_product_ID
1,[tempdb],[dbo],[#HISTWORK],PNO
0,________,________,________,________
0,[inventory_new],[jbhardware],[inventoryFactCurrent],invc_product_ID
1,[tempdb],[dbo],[#HISTWORK],PNO
0,________,________,________,________
0,[inventory_new],[shared],[retailStore],rs_store_ID
1,[inventory_new],[jbhardware],[inventoryFactCurrent],invc_store_ID


In [ ]:
{'product': {'retailSalesStoreItem_JB': {'prd_id': 'irssi_product_ID'},
  'inventoryFactCurrent': {'prd_id': 'invc_product_ID'},
  'pricing': {'prd_id': 'prc_product_ID'},
  'productFamilyGroupDetail': {'prd_family_ID': 'pfgd_family_ID'},
  'retailProduct': {'prd_id': 'rtlprd_product_ID'},
  'inboundRetailerInventoryDetail_JB': {'prd_id': 'invd_product_ID'},
  'inboundSalesInventory': {'prd_suppProduct_ID': 'is_retailKeycode'},
  'product': {'prd_id': 'prd_id', 'prd_class_ID': 'prd_class_ID'},
  'productCategory': {'prd_category_ID': 'pc_ID'},
  'InventoryHistoryDailyStoreItem': {'prd_id': 'ihdsi_product_ID'},
  'replenStoreItemWeeklySummary': {'prd_id': 'rsiws_product_ID'},
  'rangeStoreItem': {'prd_id': 'rngsi_product_ID'},
  'promoStoreItem': {'prd_id': 'prmsi_product_ID'},
  'replenStoreItemRecord': {'prd_id': 'rpld_product_ID'},
  'SalesHistoryDailyItem': {'prd_id': 'sdi_product_ID'}},
 'retailSalesStoreItem_JB': {'product': {'irssi_product_ID': 'prd_id'},
  'SalesHistoryDailyStoreItem': {'irssi_store_ID': 'sdsi_store_ID',
   'irssi_product_ID': 'sdsi_product_ID'},
  'SalesHistoryLTDStoreItem': {'irssi_store_ID': 'slsi_store_ID',
   'irssi_product_ID': 'slsi_product_ID'},
  'SalesHistoryDailyItem': {'irssi_product_ID': 'sdi_product_ID'},
  'SalesHistoryDailyRetailItem': {'irssi_product_ID': 'sdri_product_ID'}},
 'SalesHistoryDailyStoreItem': {'retailSalesStoreItem_JB': {'sdsi_store_ID': 'irssi_store_ID',
   'sdsi_product_ID': 'irssi_product_ID'},
  'inventoryFactCurrent': {'sdsi_product_ID': 'invc_product_ID',
   'sdsi_store_ID': 'invc_store_ID'},
  'SalesHistoryDailyStore': {'sdsi_store_ID': 'sds_store_ID'},
  'SalesHistoryDailyStoreItem': {'sdsi_retail_ID': 'sdsi_retail_ID',
   'sdsi_store_ID': 'sdsi_store_ID',
   'sdsi_product_ID': 'sdsi_product_ID',
   'sdsi_date': 'sdsi_date'},
  'pricing': {'sdsi_retail_ID': 'prc_retail_ID',
   'sdsi_product_ID': 'prc_product_ID'}},
 'inventoryFactCurrent': {'product': {'invc_product_ID': 'prd_id'},
  'pricing': {'invc_product_ID': 'prc_product_ID',
   'invc_retail_ID': 'prc_retail_ID'},
  'productReplacement': {'invc_product_ID': 'prp_product_ID'},
  'rangeDerived': {'invc_store_ID': 'rngd_store_ID',
   'invc_product_ID': 'rngd_product_ID'},
  'retailProduct': {'invc_product_ID': 'rtlprd_product_ID'},
  'inventoryFactCurrent': {'invc_product_ID': 'invc_product_ID'},
  'SalesHistoryDailyStoreItem': {'invc_product_ID': 'sdsi_product_ID',
   'invc_store_ID': 'sdsi_store_ID'},
  'inboundRetailerInventoryDetail_JB': {'invc_product_ID': 'invd_product_ID',
   'invc_store_ID': 'invd_store_ID'}},
 'pricing': {'product': {'prc_product_ID': 'prd_id'},
  'pricing': {'prc_startDate': 'prc_startDate',
   'prc_product_ID': 'prc_product_ID',
   'prc_retail_ID': 'prc_retail_ID'},
  'inventoryFactCurrent': {'prc_product_ID': 'invc_product_ID',
   'prc_retail_ID': 'invc_retail_ID'},
  'retailProduct': {'prc_product_ID': 'rtlprd_product_ID',
   'prc_retail_ID': 'rtlprd_retail_ID'},
  'SalesHistoryDailyStoreItem': {'prc_retail_ID': 'sdsi_retail_ID',
   'prc_product_ID': 'sdsi_product_ID'},
  'inboundSalesInventory': {'prc_supplierProduct_ID': 'is_retailKeycode'}},
 'productFamilyGroupDetail': {'productFamilyGroup': {'pfgd_group_ID': 'pfg_ID'},
  'product': {'pfgd_family_ID': 'prd_family_ID'}},
 'productFamilyGroup': {'productFamilyGroupDetail': {'pfg_ID': 'pfgd_group_ID'}},
 'retailProduct': {'pricing': {'rtlprd_product_ID': 'prc_product_ID',
   'rtlprd_retail_ID': 'prc_retail_ID'},
  'inventoryFactCurrent': {'rtlprd_product_ID': 'invc_product_ID'},
  'product': {'rtlprd_product_ID': 'prd_id'},
  'lifeCycleForecastSetup': {'rtlprd_retail_ID': 'lfs_retail_ID',
   'rtlprd_lifeCycle_ID': 'lfs_lifeCycle_ID'},
  'InventoryHistoryDailyStoreItem': {'rtlprd_product_ID': 'ihdsi_product_ID',
   'rtlprd_retail_ID': 'ihdsi_retail_ID'},
  'SalesHistoryDailyItem': {'rtlprd_product_ID': 'sdi_product_ID'}},
 'productReplacement': {'inventoryFactCurrent': {'prp_product_ID': 'invc_product_ID'}},
 'SalesHistoryLTDStoreItem': {'retailSalesStoreItem_JB': {'slsi_store_ID': 'irssi_store_ID',
   'slsi_product_ID': 'irssi_product_ID'}},
 'inbtemp': {'inactiveProduct': {'Stock_Code': 'prd_suppProduct_ID'}},
 'inactiveProduct': {'inbtemp': {'prd_suppProduct_ID': 'Stock_Code'}},
 'rangeStoreItem': {'range': {'rngsi_range_ID': 'rng_ID'},
  'product': {'rngsi_product_ID': 'prd_id'}},
 'range': {'rangeStoreItem': {'rng_ID': 'rngsi_range_ID'},
  'rangeItem': {'rng_ID': 'ritm_range_ID'},
  'storeGroupCluster': {'rng_storeGroup_ID': 'sgc_group_ID',
   'rng_iteration': 'sgc_iteration',
   'rng_storeGroupCluster': 'sgc_grade_ID'}},
 'rangeDerived': {'inventoryFactCurrent': {'rngd_store_ID': 'invc_store_ID',
   'rngd_product_ID': 'invc_product_ID'}},
 'SalesHistoryDailyStore': {'SalesHistoryDailyStoreItem': {'sds_store_ID': 'sdsi_store_ID'}},
 'inboundRetailerInventoryDetail_JB': {'inboundRetailerInventoryDetail_JB': {'invd_retail_ID': 'invd_retail_ID',
   'invd_activityDate': 'invd_activityDate',
   'invd_controlNumber': 'invd_controlNumber',
   'invd_subControlNumber': 'invd_subControlNumber',
   'invd_inboundRetailKeycode': 'invd_inboundRetailKeycode',
   'invd_inboundRetailBarcode': 'invd_inboundRetailBarcode',
   'invd_retailStore_ID': 'invd_retailStore_ID'},
  'inventoryFactCurrent': {'invd_product_ID': 'invc_product_ID',
   'invd_store_ID': 'invc_store_ID'},
  'product': {'invd_product_ID': 'prd_id'}},
 'inboundSalesInventory': {'product': {'is_retailKeycode': 'prd_suppProduct_ID'},
  'pricing': {'is_retailKeycode': 'prc_supplierProduct_ID'}},
 'productCategory': {'product': {'pc_ID': 'prd_category_ID'}},
 'deliveryPath': {'deliveryPath': {'dp_src_retailerOrSupplier': 'dp_dest_retailerOrSupplier',
   'dp_src_ID': 'dp_dest_ID',
   'dp_src_level': 'dp_dest_level',
   'dp_dest_level': 'dp_src_level'}},
 'SalesHistoryDailyItem': {'retailSalesStoreItem_JB': {'sdi_product_ID': 'irssi_product_ID'},
  'retailProduct': {'sdi_product_ID': 'rtlprd_product_ID'},
  'product': {'sdi_product_ID': 'prd_id'}},
 'lifeCycleForecastSetup': {'retailProduct': {'lfs_retail_ID': 'rtlprd_retail_ID',
   'lfs_lifeCycle_ID': 'rtlprd_lifeCycle_ID'}},
 'SalesHistoryDailyRetailItem': {'retailSalesStoreItem_JB': {'sdri_product_ID': 'irssi_product_ID'}},
 'InventoryHistoryDailyStoreItem': {'product': {'ihdsi_product_ID': 'prd_id'},
  'retailProduct': {'ihdsi_product_ID': 'rtlprd_product_ID',
   'ihdsi_retail_ID': 'rtlprd_retail_ID'}},
 'replenStoreItemWeeklySummary': {'product': {'rsiws_product_ID': 'prd_id'}},
 'promoStoreItem': {'product': {'prmsi_product_ID': 'prd_id'}},
 'rangeItem': {'range': {'ritm_range_ID': 'rng_ID'}},
 'storeGroupCluster': {'range': {'sgc_group_ID': 'rng_storeGroup_ID',
   'sgc_iteration': 'rng_iteration',
   'sgc_grade_ID': 'rng_storeGroupCluster'},
  'storeGroup': {'sgc_retail_ID': 'sg_retail_ID',
   'sgc_group_ID': 'sg_ID',
   'sgc_iteration': 'sg_iteration'}},
 'storeGroup': {'storeGroupCluster': {'sg_retail_ID': 'sgc_retail_ID',
   'sg_ID': 'sgc_group_ID',
   'sg_iteration': 'sgc_iteration'}},
 'replenStoreItemRecord': {'product': {'rpld_product_ID': 'prd_id'}}}

0        <ShowPlanXML xmlns="http://schemas.microsoft.c...
1        <ShowPlanXML xmlns="http://schemas.microsoft.c...
2        <ShowPlanXML xmlns="http://schemas.microsoft.c...
3                                                      NaN
4        <ShowPlanXML xmlns="http://schemas.microsoft.c...
                               ...                        
20697    <ShowPlanXML xmlns="http://schemas.microsoft.c...
20698    <ShowPlanXML xmlns="http://schemas.microsoft.c...
20699    <ShowPlanXML xmlns="http://schemas.microsoft.c...
20700    <ShowPlanXML xmlns="http://schemas.microsoft.c...
20701    <ShowPlanXML xmlns="http://schemas.microsoft.c...
Name: 1, Length: 20702, dtype: object

{
    "productReplacement": {
        "rangeItem": {
            "prp_customer_ID": "ritm_customer_ID",
            "prp_replacementProduct_ID": "ritm_product_ID",
            "prp_retail_ID": "ritm_retail_ID"
        }
    },
    "rangeItem": {
        "productReplacement": {
            "ritm_customer_ID": "prp_customer_ID",
            "ritm_product_ID": "prp_replacementProduct_ID",
            "ritm_retail_ID": "prp_retail_ID"
        }
    }
}


In [24]:
xml_list = df.iloc[11758,1].replace('xmlns="http://schemas.microsoft.com/sqlserver/2004/07/showplan" ','')
def xml_relationship_parsing(xml_list):
    tree = ET.ElementTree(ET.fromstring(xml_list))
    root = tree.getroot()[0][0]
    df_column = pd.DataFrame(columns=["Database", "Schema", "Table", "Column"])
    empty_df = pd.DataFrame(columns=["Database", "Schema", "Table", "Column"],data=[['________']*4])
    t1=[[c1.attrib,c2.find('.//ColumnReference').attrib] for a in root.findall('.//SeekPredicates') for c1,c2 in zip(a.findall('.//RangeColumns/ColumnReference'),a.findall('.//RangeExpressions//ScalarOperator')) if c2[0].tag!='Const']
    t2=[[b.attrib for b in a.findall('.//ColumnReference')] for a in root.findall('.//ProbeResidual//Compare')+root.findall('.//Residual//Compare')]
    t=t1+t2
    relationship={}
    for column1,column2 in t:
        try:
            if column1['Schema'] =='[jbhardware]' and column2['Schema'] =='[jbhardware]':
                current_relationship={
                    column1['Table'][1:-1]:{
                        column2['Table'][1:-1]:{
                            column1['Column']:column2['Column']
                        }
                    },
                    column2['Table'][1:-1]:{
                        column1['Table'][1:-1]:{
                            column2['Column']:column1['Column']
                        }
                    }
                }
                relationship=merge(relationship, current_relationship, path=None)
    #                 relationship[column1['Table'][1:-1]][column2['Table'][1:-1]][column1['Column']]=column2['Column']
    #                 relationship[column2['Table'][1:-1]][column1['Table'][1:-1]][column2['Column']]=column1['Column']
        except:
            continue
    return relationship
#     return t

result = xml_relationship_parsing(xml_list)
print(json.dumps(result, indent=4, sort_keys=True))

{
    "productReplacement": {
        "rangeItem": {
            "prp_customer_ID": "ritm_customer_ID",
            "prp_replacementProduct_ID": "ritm_product_ID",
            "prp_retail_ID": "ritm_retail_ID"
        }
    },
    "rangeItem": {
        "productReplacement": {
            "ritm_customer_ID": "prp_customer_ID",
            "ritm_product_ID": "prp_replacementProduct_ID",
            "ritm_retail_ID": "prp_retail_ID"
        }
    }
}
